In [1]:
from dm_control import mjcf
from dm_control import viewer
from dm_control import mujoco
from dm_control import suite
import numpy as np
import re

## Define useful parameters and functions

In [2]:
base_model_path = '/home/diego/code/olveczky/dm/stac/models/to_tune/rat_may17.xml'
# temp_model_path is the model used by .../dm_control/suite/rat.py for easy iteration.
temp_model_path = '/home/diego/.envs/mujoco200_3.7/lib/python3.6/site-packages/dm_control/suite/rat_temp.xml'

def view_model():
    # Load an environment from the Control Suite.
    env = suite.load(domain_name="rat", task_name="stand")
    # Launch the viewer application.
    viewer.launch(env)

def load_model(model_path):
    return mjcf.from_path(model_path)

def write_model(model, save_path):
    s = model.to_xml_string()
    s = re.sub('rat_skin.*skn', 'rat_skin.skn', s)
    with open(save_path , 'w') as f:
        f.write(s)

# Save measured data (mm) and the bone-site pairs they correspond to 
bone_lengths = {'humerus': 30.0,
                'radius': 29.6,
                'femur': 36.5,
                'tibia': 42.8,
                'metatarsal': 23.4}
joint_pairs = {'humerus': ['shoulder_L', 'elbow_L'],
               'radius': ['elbow_L', 'wrist_L'],
               'femur': ['hip_L', 'knee_L'],
               'tibia': ['knee_L', 'ankle_L'],
               'metatarsal': ['ankle_L', 'toe_L']}

base_model = load_model(base_model_path)

In [3]:
def get_bone_distance(physics, joint_pair):
    joint0 = physics.named.data.site_xpos[joint_pair[0]].copy()
    joint1 = physics.named.data.site_xpos[joint_pair[1]].copy()
    length = np.sqrt(np.sum((joint0 - joint1)**2))
    return length

def get_bone_ratios(bone_dict):
    n_bones = len(bone_dict.keys())
    ratio_mat = np.zeros((n_bones,n_bones))
    ratio_dict = {}
    for i, (bone0, length0) in enumerate(bone_dict.items()):
        for j, (bone1, length1) in enumerate(bone_dict.items()):
            ratio = length0/length1
            ratio_mat[i, j] = ratio
            ratio_dict[bone0 + '-' + bone1] = ratio
            
    return ratio_dict, ratio_mat

In [4]:
# Globally scale down the model
def scale_model(model, global_scale_ratio=.82):
    for g in model.find_all('geom'):
        if g.pos is not None and 'eye' not in g.name:
            g.pos *= global_scale_ratio
    for b in model.find_all('body'):
        if b.pos is not None and 'eye' not in g.name:
            b.pos *= global_scale_ratio
    for s in model.find_all('site'):
        if s.pos is not None and 'eye' not in g.name:
            s.pos *= global_scale_ratio
    return model
model = scale_model(base_model)
write_model(model, temp_model_path)

# Scale particular arm and leg joints to match 
def scale_arms_and_legs(model):
    env = suite.load(domain_name="rat", task_name="stand")
    model_lengths = {k: get_bone_distance(env.physics, jp)*1000 for k, jp in joint_pairs.items()}
    ratio = [bone_lengths[k]/model_lengths[k] for k in model_lengths.keys()]
    model_name_pairs = {'humerus': 'elbow',
                   'radius': 'wrist',
                   'femur': 'knee',
                   'tibia': 'ankle',
                   'metatarsal': 'toe'}
    for i, (bone, model_id) in enumerate(model_name_pairs.items()):
        for g in model.find_all('geom'):
            if model_name_pairs[bone] in g.name:
                g.pos *= ratio[i]
        for b in model.find_all('body'):
            if model_name_pairs[bone] in b.name:
                b.pos *= ratio[i]
        for s in model.find_all('site'):
            if model_name_pairs[bone] in s.name:
                s.pos *= ratio[i]
    return model
model = scale_arms_and_legs(model)
write_model(model, temp_model_path)

In [5]:
env = suite.load(domain_name="rat", task_name="stand")
model_lengths = {k: get_bone_distance(env.physics, jp)*1000 for k, jp in joint_pairs.items()}
length_difference = {k: bone_lengths[k] - model_lengths[k] for k in model_lengths.keys()}
bone_ratios, bone_ratio_mat = get_bone_ratios(bone_lengths)
model_ratios, model_ratio_mat = get_bone_ratios(model_lengths)
print(model_lengths)
print(bone_lengths)
ratio = [bone_lengths[k]/model_lengths[k] for k in model_lengths.keys()]
print(ratio)

{'humerus': 30.0, 'radius': 29.63997992437027, 'femur': 36.5, 'tibia': 42.88676441519942, 'metatarsal': 23.2895915800861}
{'humerus': 30.0, 'radius': 29.6, 'femur': 36.5, 'tibia': 42.8, 'metatarsal': 23.4}
[1.0, 0.9986511487365282, 1.0, 0.9979768952873331, 1.0047406765178444]


In [6]:
view_model()